In [ ]:
%%shell
# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium
pip install --upgrade selenium


Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

movie_info = [
    ('https://www.imdb.com/title/tt5971474/reviews', 'The Little Mermaid'),
    ('https://www.imdb.com/title/tt5635026/reviews', 'Peter Pan & Wendy'),
    ('https://www.imdb.com/title/tt2527336/reviews', 'Star Wars: The Last Jedi'),
    ('https://www.imdb.com/title/tt4154664/reviews', 'Captain Marvel')
]

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

titles = []
contents = []
ratings = []
dates = []
user_names = []
movie_titles = []

for movie_url, movie_title in movie_info:
    driver.get(movie_url)

    for i in range(5000):
        try:
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
            time.sleep(2)  # Small delay to allow the reviews to load before proceeding
        except:
            pass

    reviews = driver.find_elements(By.CSS_SELECTOR, '.lister-item-content')

    for review in reviews:
        try:
            # Extracting the title, content, rating, date, and username for each review
            review_title = review.find_element(By.CSS_SELECTOR, '.title').text
            review_content = review.find_element(By.CSS_SELECTOR, '.text').text
            review_rating = review.find_element(By.CSS_SELECTOR, '.rating-other-user-rating').text
            review_date = review.find_element(By.CSS_SELECTOR, '.review-date').text
            review_user = review.find_element(By.CSS_SELECTOR, '.display-name-link').text

            # Appending to lists
            titles.append(review_title)
            contents.append(review_content)
            ratings.append(review_rating)
            dates.append(review_date)
            user_names.append(review_user)
            movie_titles.append(movie_title)
        except:
            continue

# Build a data dictionary
data = {
    'Movie_title': movie_titles,
    'User_name': user_names,
    'Review_title': titles,
    'Review_rating': ratings,
    'Review_date': dates,
    'Review_body': contents
}


review_df = pd.DataFrame(data)

review_df.to_csv('movie_reviews.csv', index=False)

# Close the webdriver
driver.quit()



<ipython-input-7-14643b8c5f40>:19: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-7-14643b8c5f40>:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
